# Darlington Pairs — TIP120, Cascaded Gain
## When One Transistor Is Not Enough

**Module 03 — Transistors (BJT) | Notebook 02**

---

Sometimes a single BJT's current gain (beta) is not enough. If you need to switch a 2A
motor from a microcontroller that can only source 1 mA, you need a total gain of 2000 —
far beyond any single BJT. The **Darlington pair** solves this by cascading two BJTs.

**What you will learn:**
- Why single BJT beta is sometimes insufficient
- How a Darlington pair multiplies current gain
- The TIP120: an integrated Darlington in a TO-220 package
- The tradeoffs: higher V_BE(sat), higher V_CE(sat), slower switching
- Flyback diodes for inductive loads
- When to use a Darlington vs when a MOSFET is better (preview)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import ipywidgets as widgets
from IPython.display import display, HTML

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

---
## Concept — The Problem: Beta Is Not Always Enough

Consider this scenario:

- You need to switch a **12V DC motor** that draws **1.5A**
- Your microcontroller GPIO pin can source at most **1 mA** (some are even less)
- Required current gain: 1500 / 1 = **1500**

A 2N3904 has beta = 100-300. Even at the high end, 300 x 1 mA = 300 mA — not enough.
And the 2N3904 can only handle 200 mA anyway!

**Solution: Cascade two transistors.** The first transistor's collector current becomes
the second transistor's base current. Total gain = beta1 x beta2.

If beta1 = beta2 = 100: total gain = 100 x 100 = **10,000**. That is more than enough.

In [ ]:
# Compare effective beta of single BJT vs Darlington pair

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: Current gain comparison
beta_single_range = np.arange(50, 401, 10)

beta_darlington = beta_single_range ** 2  # beta_total = beta1 * beta2

ax1.semilogy(beta_single_range, beta_single_range, 'b-', linewidth=2.5, label='Single BJT')
ax1.semilogy(beta_single_range, beta_darlington, 'r-', linewidth=2.5, label='Darlington pair')

# Mark typical values
ax1.axhline(y=200, color='blue', linestyle=':', alpha=0.5)
ax1.axhline(y=200**2, color='red', linestyle=':', alpha=0.5)
ax1.plot(200, 200, 'bo', markersize=10, markeredgecolor='black', markeredgewidth=2)
ax1.plot(200, 200**2, 'ro', markersize=10, markeredgecolor='black', markeredgewidth=2)

ax1.annotate(f'beta = 200', xy=(200, 200), xytext=(250, 120),
             fontsize=11, arrowprops=dict(arrowstyle='->', color='blue'),
             color='blue', fontweight='bold')
ax1.annotate(f'beta = 40,000', xy=(200, 40000), xytext=(250, 15000),
             fontsize=11, arrowprops=dict(arrowstyle='->', color='red'),
             color='red', fontweight='bold')

ax1.set_xlabel('Individual Transistor Beta', fontsize=12)
ax1.set_ylabel('Effective Current Gain (log scale)', fontsize=12)
ax1.set_title('Current Gain: Single BJT vs Darlington', fontsize=13, fontweight='bold')
ax1.legend(fontsize=12)
ax1.set_ylim(10, 200000)

# Right: What current can you control?
I_B_available = np.array([0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 5.0])  # mA from source

beta_single = 200
beta_darlington_typ = 200 * 200  # 40000
# TIP120 is typically 1000 min (spec'd as a pair)
beta_tip120 = 1000

I_C_single = I_B_available * beta_single  # mA
I_C_darlington = I_B_available * beta_tip120  # mA, using TIP120 spec

x = np.arange(len(I_B_available))
width = 0.35

bars1 = ax2.bar(x - width/2, I_C_single, width, label=f'Single BJT (beta={beta_single})', 
                color='#3498db', edgecolor='black')
bars2 = ax2.bar(x + width/2, np.minimum(I_C_darlington, 5000), width, 
                label=f'TIP120 Darlington (beta={beta_tip120})', 
                color='#e74c3c', edgecolor='black')

ax2.set_xlabel('Available Base Current (mA)', fontsize=12)
ax2.set_ylabel('Controllable Collector Current (mA)', fontsize=12)
ax2.set_title('Current You Can Control', fontsize=13, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels([f'{v}' for v in I_B_available])
ax2.legend(fontsize=10)
ax2.set_yscale('log')

# TIP120 max current line
ax2.axhline(y=5000, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax2.text(6.5, 5500, 'TIP120 max: 5A', fontsize=9, color='red', ha='right')

# 2N3904 max current line
ax2.axhline(y=200, color='blue', linestyle='--', linewidth=1.5, alpha=0.7)
ax2.text(6.5, 230, '2N3904 max: 200mA', fontsize=9, color='blue', ha='right')

plt.tight_layout()
plt.show()

---
## Concept — Darlington Pair Internal Structure

A Darlington pair connects two BJTs so that the emitter of the first drives the base
of the second:

```
                   C (Collector — both collectors tied together)
                   |
              +----+----+
              |         |
              C         |
    B ---| Q1 (driver)  |
              E         |
              |         |
              B         |
              | Q2 (output)
              E         |
              |         |
              +---------+
              |
              E (Emitter)
```

### Total Current Gain

$$\beta_{total} = \beta_1 \times \beta_2 + \beta_1 + \beta_2 \approx \beta_1 \times \beta_2$$

The approximation holds because beta1 * beta2 >> beta1 + beta2.

### The TIP120: Integrated Darlington NPN

The **TIP120** is a popular integrated Darlington NPN transistor:

```
    TIP120 Internal Structure:
    
    B ---[R1 = 8k]---+--- B(Q1)
                      |      |
                      |      E --- B(Q2)
                      |             |
               [R2 = 120]          E ---+--- E (pin)
                      |                 |
                      +---GND (via E)---+
    
    C(Q1) and C(Q2) are tied together -> C (pin)
    Built-in flyback diode from E to C
```

- **R1 (8k ohm)**: Ensures Q1 turns off cleanly when base drive is removed
- **R2 (120 ohm)**: Ensures Q2 turns off cleanly (bleeds stored base charge)
- **Built-in flyback diode**: For driving inductive loads like motors and relays
- **TO-220 package**: Can dissipate up to 65W with a heatsink

---
## The Material Science Why — Two Junctions in Series

The Darlington pair has a significant consequence at the silicon level:

Since the emitter of Q1 drives the base of Q2, the total base-emitter voltage
is TWO PN junctions in series:

$$V_{BE(total)} = V_{BE(Q1)} + V_{BE(Q2)} \approx 0.7V + 0.7V = 1.4V$$

And when saturated:

$$V_{CE(sat)} \approx V_{CE(sat,Q2)} + V_{BE(Q2)} \approx 0.2V + 0.7V = 0.9V \text{ (minimum)}$$

For the TIP120 specifically, the datasheet says **V_CE(sat) ~ 2V** at 3A. This is much
higher than a single transistor's 0.2V.

**Why is this a problem?** Power dissipation:
- At I_C = 3A and V_CE(sat) = 2V: P = 2V x 3A = **6 watts!**
- That is a LOT of heat for a transistor that is supposedly "fully on"

This is the main disadvantage of Darlington pairs and the reason MOSFETs (covered in
Module 04) have largely replaced them for power switching.

In [ ]:
# Calculate power dissipation in TIP120 vs load power

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

V_supply = 12.0  # volts
I_load = np.linspace(0.01, 5.0, 200)  # amps

# TIP120: V_CE(sat) depends on current (approximate from datasheet)
# Roughly: V_CE(sat) ~ 1.0V at low current, rising to ~2-4V at high current
V_CE_sat_tip120 = 1.0 + 0.3 * I_load  # simplified model from datasheet curves

# Single BJT (like TIP31C): V_CE(sat) ~ 0.2-1.0V
V_CE_sat_single = 0.2 + 0.1 * I_load

# Power dissipated in transistor
P_tip120 = V_CE_sat_tip120 * I_load
P_single = V_CE_sat_single * I_load

# Power delivered to load
P_load_tip120 = (V_supply - V_CE_sat_tip120) * I_load
P_load_single = (V_supply - V_CE_sat_single) * I_load

# Left plot: Power dissipation comparison
ax1.plot(I_load, P_tip120, 'r-', linewidth=2.5, label='TIP120 (Darlington)')
ax1.plot(I_load, P_single, 'b-', linewidth=2.5, label='Single power BJT')
ax1.axhline(y=65, color='red', linestyle='--', alpha=0.5, label='TIP120 max (with heatsink)')
ax1.axhline(y=2, color='orange', linestyle='--', alpha=0.5, label='TIP120 max (no heatsink)')

ax1.set_xlabel('Load Current (A)', fontsize=12)
ax1.set_ylabel('Power Dissipated in Transistor (W)', fontsize=12)
ax1.set_title('Power Wasted in the Transistor', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.set_xlim(0, 5)
ax1.set_ylim(0, 20)

# Right plot: Efficiency comparison
eff_tip120 = P_load_tip120 / (P_load_tip120 + P_tip120) * 100
eff_single = P_load_single / (P_load_single + P_single) * 100

ax2.plot(I_load, eff_tip120, 'r-', linewidth=2.5, label='TIP120 (Darlington)')
ax2.plot(I_load, eff_single, 'b-', linewidth=2.5, label='Single power BJT')

# MOSFET preview (very low R_DS(on))
R_DS_on = 0.04  # 40 milliohms for a decent MOSFET
V_mosfet = I_load * R_DS_on
P_mosfet = V_mosfet * I_load
P_load_mosfet = (V_supply - V_mosfet) * I_load
eff_mosfet = P_load_mosfet / (P_load_mosfet + P_mosfet) * 100
ax2.plot(I_load, eff_mosfet, 'g--', linewidth=2.5, label='MOSFET (preview, R_DS=40mohm)')

ax2.set_xlabel('Load Current (A)', fontsize=12)
ax2.set_ylabel('Efficiency (%)', fontsize=12)
ax2.set_title('Switching Efficiency (12V supply)', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.set_xlim(0, 5)
ax2.set_ylim(50, 100)

plt.tight_layout()
plt.show()

print("At 3A load current (12V supply):")
idx_3A = np.argmin(np.abs(I_load - 3.0))
print(f"  TIP120 Darlington: {eff_tip120[idx_3A]:.1f}% efficient, wastes {P_tip120[idx_3A]:.1f}W")
print(f"  Single power BJT:  {eff_single[idx_3A]:.1f}% efficient, wastes {P_single[idx_3A]:.1f}W")
print(f"  MOSFET (preview):  {eff_mosfet[idx_3A]:.1f}% efficient, wastes {P_mosfet[idx_3A]:.1f}W")
print(f"\nThe MOSFET advantage is clear — we cover this in Module 04!")

---
## Concept — Flyback Diode for Inductive Loads

When switching **inductive loads** (motors, relays, solenoids), you MUST use a flyback
diode. Here is why:

An inductor resists changes in current. When you turn OFF the transistor, the inductor
tries to keep current flowing. With nowhere to go, it generates a massive voltage spike
(potentially hundreds of volts!) that can destroy your transistor.

```
    V_supply (+12V)
         |                        V_supply (+12V)
      [Motor]                          |
    +----+----+                     [Motor]
    |         |                   +----+----+
   (Diode)    |        ==>       |  D  |    |
    |    C    |                   (1N4001)   |
    +--| TIP120                   |    C    |
       B     E                    +--| TIP120
       |     |                       B     E
    [R_B]   GND                      |     |
       |                          [R_B]   GND
    Signal                           |
                                  Signal
    WITHOUT flyback diode         WITH flyback diode
    (DANGEROUS!)                  (SAFE)
```

The **TIP120 has a built-in flyback diode**, but it is a good practice to add an external
one (like a 1N4001) for extra protection, especially with large inductive loads.

The diode is placed **reverse-biased** across the load (cathode to V+, anode to collector).
During normal operation it does nothing. When the transistor turns off, the inductor's
voltage spike forward-biases the diode, providing a safe path for the current to decay.

---
## Concept — When Darlington vs When MOSFET

| Feature | Darlington (TIP120) | MOSFET (e.g., IRLZ44N) |
|---------|--------------------|-----------------------|
| Input | Current-driven (needs mA) | Voltage-driven (needs volts, ~0 current) |
| V_CE(sat) / V_DS(on) | ~1-4V (high!) | ~0.01-0.1V (extremely low) |
| Switching speed | Slow (charge storage) | Fast (no minority carriers) |
| Gate/base drive | Simple — just a resistor | Needs voltage level, sometimes driver |
| Power waste when ON | Significant (V_CE * I_C) | Very low (I^2 * R_DS) |
| Cost | Very cheap | Cheap (used to be expensive) |
| Availability | Everywhere | Everywhere |

**Use a Darlington when:**
- You need extreme simplicity (just add a base resistor)
- Current is low (<500 mA) and efficiency does not matter
- You already have one in your parts bin
- You are working with a legacy design

**Use a MOSFET when:**
- Efficiency matters (almost always)
- Load current is significant (>500 mA)
- Switching speed matters
- Power dissipation or heat is a concern

**In practice**: For new designs, MOSFETs have almost entirely replaced Darlingtons.
We cover MOSFETs in detail in Module 04.

In [ ]:
# Plot efficiency vs load current for Darlington driver at different supply voltages

fig, ax = plt.subplots(figsize=(11, 6))

I_load = np.linspace(0.05, 5.0, 200)

supply_voltages = [5, 9, 12, 24]
colors = ['#e74c3c', '#e67e22', '#3498db', '#2ecc71']

for V_s, color in zip(supply_voltages, colors):
    # TIP120 V_CE(sat) model
    V_CE_sat = 1.0 + 0.3 * I_load
    
    # Efficiency
    P_load = (V_s - V_CE_sat) * I_load
    P_total = V_s * I_load
    efficiency = np.where(P_load > 0, P_load / P_total * 100, 0)
    
    ax.plot(I_load, efficiency, color=color, linewidth=2.5, label=f'V_supply = {V_s}V')

ax.set_xlabel('Load Current (A)', fontsize=13)
ax.set_ylabel('Efficiency (%)', fontsize=13)
ax.set_title('TIP120 Darlington — Efficiency vs Load Current\n'
             '(Higher supply voltage = less wasted fraction)',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=12)
ax.set_xlim(0, 5)
ax.set_ylim(0, 100)

# Annotation
ax.annotate('At 5V supply and 3A:\nV_CE(sat) ~ 1.9V\n= 38% wasted!',
            xy=(3, 62), xytext=(3.5, 40),
            fontsize=11, fontweight='bold', color='#e74c3c',
            arrowprops=dict(arrowstyle='->', color='#e74c3c', lw=2),
            bbox=dict(boxstyle='round', facecolor='lightyellow'))

ax.annotate('At 24V supply:\nV_CE(sat) is small\nfraction of total',
            xy=(3, 92), xytext=(0.5, 75),
            fontsize=11, fontweight='bold', color='#2ecc71',
            arrowprops=dict(arrowstyle='->', color='#2ecc71', lw=2),
            bbox=dict(boxstyle='round', facecolor='lightyellow'))

plt.tight_layout()
plt.show()

print("Key takeaway: Darlington pairs are most efficient at HIGHER supply voltages")
print("where V_CE(sat) is a small fraction of the total. At low voltages (3.3V, 5V),")
print("the voltage wasted across the Darlington becomes unacceptable.")

---
## Experiment 1 — TIP120 Driving LEDs from a Low-Current Signal

### Equipment
- TIP120 Darlington NPN transistor (TO-220 package)
- Bench power supply: 12V for LED strip / load
- Second supply output or divider: simulates a weak control signal
- Klein MM300 multimeter
- LEDs (3-4 in parallel, each with its own current-limiting resistor)

### Circuit

```
    V_CC (+12V bench supply)
         |
         +---[220 ohm]---LED1---+
         |                      |
         +---[220 ohm]---LED2---+
         |                      |
         +---[220 ohm]---LED3---+
         |                      |
         +---[220 ohm]---LED4---+
                                |
                                C
    V_in (5V) ---[1k]--- B ---| TIP120
                                E
                                |
                               GND
    
    Pinout (TO-220, looking at the front with tab at top):
    Left pin:   Base (B)
    Middle pin: Collector (C)
    Right pin:  Emitter (E)
```

### Procedure

1. Build the circuit on a breadboard
2. Set V_CC to 12V, V_in to 5V
3. All four LEDs should light up brightly

### Measurements

With Klein MM300 in DC voltage mode:

| Measurement | Expected | Your Value |
|:---|:---:|:---:|
| V_BE (base to emitter) | 1.2-1.4V (two junctions!) | _____ |
| V_CE(sat) (collector to emitter) | 1.0-2.0V | _____ |
| V across each 220-ohm resistor | ~(12-2-V_CE)/220 * 220 | _____ |

Compare to a 2N3904 (swap it in with only one LED):

| Measurement | TIP120 | 2N3904 |
|:---|:---:|:---:|
| V_BE | ~1.3V | ~0.7V |
| V_CE(sat) | ~1.0-2.0V | ~0.1-0.3V |

---
## Experiment 2 — Measuring V_CE(sat) at Different Load Currents

This experiment quantifies the TIP120's main disadvantage.

### Circuit

```
    V_CC (+12V bench supply)
         |
        [R_load]  (swap different values to change I_C)
         |
         C
    5V ---[1k]--- B ---| TIP120
                        E
                        |
                       GND
```

### Procedure

Swap R_load and measure V_CE(sat) each time. The bench supply current display gives I_C.

| R_load | Expected I_C | Measured V_CE(sat) | P_waste = V_CE * I_C |
|:---:|:---:|:---:|:---:|
| 1k ohm | ~11 mA | _____ | _____ |
| 220 ohm | ~50 mA | _____ | _____ |
| 100 ohm | ~110 mA | _____ | _____ |
| 47 ohm | ~230 mA | _____ | _____ |
| 22 ohm | ~490 mA | _____ | _____ |
| 10 ohm | ~1A | _____ | _____ |

In [ ]:
# Fill in your measurements here and plot them!

# Replace None with your measured values
R_loads = [1000, 220, 100, 47, 22, 10]  # ohms
I_C_measured = [None, None, None, None, None, None]  # amps (from bench supply display)
V_CE_sat_measured = [None, None, None, None, None, None]  # volts

# Check if measurements are filled in
if all(v is not None for v in V_CE_sat_measured):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    I_C_mA = [i * 1000 for i in I_C_measured]
    P_waste = [v * i for v, i in zip(V_CE_sat_measured, I_C_measured)]
    
    ax1.plot(I_C_mA, V_CE_sat_measured, 'ro-', linewidth=2, markersize=8, label='Your measurements')
    ax1.set_xlabel('I_C (mA)', fontsize=12)
    ax1.set_ylabel('V_CE(sat) (V)', fontsize=12)
    ax1.set_title('TIP120 V_CE(sat) vs Current', fontsize=13, fontweight='bold')
    ax1.legend(fontsize=11)
    
    ax2.plot(I_C_mA, [p * 1000 for p in P_waste], 'bo-', linewidth=2, markersize=8, label='Your measurements')
    ax2.set_xlabel('I_C (mA)', fontsize=12)
    ax2.set_ylabel('Power Wasted (mW)', fontsize=12)
    ax2.set_title('TIP120 Power Dissipation', fontsize=13, fontweight='bold')
    ax2.legend(fontsize=11)
    
    plt.tight_layout()
    plt.show()
else:
    print("Fill in your measurements above and re-run this cell!")
    print("\nExpected approximate V_CE(sat) values for TIP120:")
    print("  At ~10 mA:  ~0.8-1.0 V")
    print("  At ~50 mA:  ~0.9-1.2 V")
    print("  At ~100 mA: ~1.0-1.5 V")
    print("  At ~250 mA: ~1.2-1.8 V")
    print("  At ~500 mA: ~1.5-2.0 V")
    print("  At ~1 A:    ~2.0-3.0 V")
    print("\nCompare to 2N3904: V_CE(sat) ~ 0.1-0.3V at similar currents!")

---
## Simulation — Build It in Falstad

Build a Darlington pair to see how two transistors multiply their current gain.

**What to build:**

1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Place two NPN transistors (right-click → **Draw** → **Active Components** → **Add NPN Transistor**)
3. Wire the **emitter** of the first transistor to the **base** of the second transistor — this is the Darlington connection
4. Connect both **collectors** together, then through a load (LED + 220Ω resistor) to a 9V supply
5. Connect the **emitter** of the second transistor to ground
6. Connect a very large resistor (1 MΩ) from a voltage source to the **base** of the first transistor

**What to observe:**
- Even with 1 MΩ base resistance, enough current reaches the load to light the LED — a single transistor couldn't do this
- The overall current gain is β₁ × β₂ — typically thousands
- Hover over the base wire of the first transistor: tiny current. Hover over the collector: much larger current.
- The base-emitter voltage is now ~1.4V (two V_BE drops in series) instead of the usual ~0.7V

**What to try:**
- Replace the Darlington pair with a single transistor and the same 1 MΩ base resistor — the LED won't light (not enough base current for a single stage)
- Try varying the base input and observe how tiny base currents control large collector currents

---
## Datasheet Connection — TIP120

Key parameters from the TIP120 datasheet:

| Parameter | Value | Significance |
|-----------|-------|--------------|
| V_CEO (max) | 60V | Maximum collector-emitter voltage |
| I_C (max) | 5A | Maximum continuous collector current |
| h_FE (min) | 1000 @ I_C=0.5A | Minimum current gain — guaranteed! |
| V_CE(sat) | 2.0V @ I_C=3A, I_B=12mA | Saturation voltage (this is the big number) |
| V_BE(on) | 2.5V @ I_C=3A | Two-junction forward voltage |
| P_D (max) | 65W (with heatsink) | Maximum power dissipation |
| P_D (free air) | ~2W | Without heatsink — the real limit! |
| Package | TO-220 | Can bolt to heatsink |
| Built-in diode | Yes | Collector-to-emitter, for inductive loads |

### Critical Reading: V_CE(sat) = 2V at 3A

This means that even when the TIP120 is fully ON, it drops 2V. At 3A:
- Power in TIP120: 2V x 3A = 6W
- If you are using a 5V supply: 6W out of 15W total = **40% wasted!**
- If you are using a 24V supply: 6W out of 72W total = only 8% wasted

This is why Darlington pairs work better at higher voltages.

In [ ]:
# TIP120 thermal analysis — can you run your circuit without a heatsink?

def tip120_thermal_check(V_supply, I_load, ambient_temp_C=25):
    """Check if a TIP120 needs a heatsink."""
    # TIP120 V_CE(sat) approximation
    V_CE_sat = 1.0 + 0.33 * I_load  # rough approximation
    
    P_diss = V_CE_sat * I_load  # watts
    
    # TIP120 thermal resistance (from datasheet)
    theta_JC = 1.92   # junction to case, C/W
    theta_JA = 62.5   # junction to ambient (free air, no heatsink), C/W
    T_J_max = 150     # max junction temperature, C
    
    # Temperature rise without heatsink
    T_J_no_hs = ambient_temp_C + P_diss * theta_JA
    
    # Max power without heatsink
    P_max_no_hs = (T_J_max - ambient_temp_C) / theta_JA
    
    # Max power with a basic heatsink (assume theta_SA ~ 10 C/W)
    theta_CS = 0.5  # case to sink (with thermal paste)
    theta_SA = 10   # sink to ambient (small heatsink)
    theta_total_hs = theta_JC + theta_CS + theta_SA
    P_max_with_hs = (T_J_max - ambient_temp_C) / theta_total_hs
    T_J_with_hs = ambient_temp_C + P_diss * theta_total_hs
    
    print("=" * 55)
    print("TIP120 THERMAL ANALYSIS")
    print("=" * 55)
    print(f"\nOperating conditions:")
    print(f"  V_supply = {V_supply}V, I_load = {I_load}A")
    print(f"  V_CE(sat) ~ {V_CE_sat:.1f}V")
    print(f"  Power dissipation = {P_diss:.2f}W")
    print(f"  Ambient temperature = {ambient_temp_C} C")
    
    print(f"\nWithout heatsink:")
    print(f"  Max power = {P_max_no_hs:.1f}W")
    print(f"  Junction temp = {T_J_no_hs:.0f} C", end="")
    if T_J_no_hs > T_J_max:
        print(f" >>> EXCEEDS {T_J_max} C LIMIT! NEEDS HEATSINK!")
    elif T_J_no_hs > 100:
        print(f" (hot! heatsink recommended)")
    else:
        print(f" (OK)")
    
    print(f"\nWith small heatsink (theta_SA = {theta_SA} C/W):")
    print(f"  Max power = {P_max_with_hs:.1f}W")
    print(f"  Junction temp = {T_J_with_hs:.0f} C", end="")
    if T_J_with_hs > T_J_max:
        print(f" >>> STILL TOO HOT! Need bigger heatsink.")
    else:
        print(f" (OK)")
    
    return P_diss, T_J_no_hs, T_J_with_hs

print("--- Scenario 1: LED strip at 500mA, 12V ---")
tip120_thermal_check(12, 0.5)

print("\n--- Scenario 2: Motor at 2A, 12V ---")
tip120_thermal_check(12, 2.0)

print("\n--- Scenario 3: Heavy load at 4A, 12V ---")
tip120_thermal_check(12, 4.0)

---
## Checkpoint Questions

**Q1.** A Darlington pair is made from two BJTs, each with beta = 150. What is the total
current gain? If the base current is 100 uA, what is the collector current?

<details><summary>Answer</summary>

beta_total = 150 x 150 = 22,500

I_C = beta_total x I_B = 22,500 x 100 uA = 2.25 A

(In practice, the TIP120 datasheet guarantees only beta >= 1000 at rated current,
because the internal transistors have lower beta at high collector currents.)
</details>

**Q2.** Why is V_BE(on) approximately 1.4V for a Darlington pair instead of the usual 0.7V?

<details><summary>Answer</summary>

The Darlington pair has two base-emitter junctions in series. The emitter of Q1
connects to the base of Q2. Each junction drops ~0.7V, so the total is ~1.4V.
</details>

**Q3.** You are driving a 12V relay coil (200 ohms) with a TIP120 from a 3.3V microcontroller
signal. Calculate R_B, I_C, and the power dissipated in the TIP120. Do you need a heatsink?

<details><summary>Answer</summary>

I_C = (12V - V_CE(sat)) / 200 ohms. With V_CE(sat) ~ 1.2V: I_C ~ 54 mA.

I_B = I_C / beta(min) = 54 mA / 1000 = 54 uA (very tiny!)

But use forced beta of ~100: I_B = 54 mA / 100 = 0.54 mA.

R_B = (3.3V - 1.4V) / 0.54 mA = 3.5k. Use 3.3k.

P_TIP120 = V_CE(sat) x I_C = 1.2V x 54 mA = 65 mW. No heatsink needed.
</details>

**Q4.** Why must you ALWAYS use a flyback diode when switching inductive loads like motors
and relays? What happens physically without one?

<details><summary>Answer</summary>

An inductor resists changes in current (V = L * dI/dt). When the transistor turns off,
dI/dt becomes very large and negative. The inductor generates a large positive voltage
spike at the collector — potentially hundreds of volts. This exceeds the transistor's
V_CEO rating and can destroy it. The flyback diode provides a safe current path for the
inductor's stored energy to dissipate through.
</details>

**Q5.** At what supply voltage does a TIP120 become "inefficient enough to avoid"?
Why does supply voltage matter?

<details><summary>Answer</summary>

Below about 5V, the TIP120's V_CE(sat) of 1-2V wastes 20-40% of the supply voltage.
At 3.3V it is nearly useless. The fraction wasted = V_CE(sat) / V_supply, so higher
supply voltages make the fixed V_CE(sat) drop a smaller percentage. At 24V supply,
even a 2V drop is only 8%. For low-voltage applications, use a MOSFET instead.
</details>